In [1]:
import sys
import os
import shutil
import gzip
import csv
import multiprocessing

import tqdm.notebook as tqnb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd

import boda
from boda.common.utils import KmerFilter, dna2tensor

import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

In [2]:
def pad_tensor_200(in_tensor):
    out_tensor = torch.cat([
        in_tensor,
        torch.zeros((4,200-in_tensor.shape[1]), device=in_tensor.device)
    ], dim=1)
    return out_tensor

In [3]:
seq_table = pd.read_table('boda_round_1_controls_20211215.seq', sep='\t', header=None, names=['ID','sequence'])

seq_tensor = torch.stack(
    [ pad_tensor_200(dna2tensor(line['sequence'])) for i, line in seq_table.iterrows() ], 
    dim=0
)


In [4]:
token_tensor = seq_tensor.argmax(dim=1)

In [7]:
section_size = torch.arange(token_tensor.shape[0]).flip(dims=[0])
flat_idxer   = torch.cat([torch.tensor([0],dtype=torch.long),torch.cumsum(section_size,dim=0,dtype=torch.long)])

edit_dist = torch.full((torch.arange(token_tensor.shape[0]).sum(),), fill_value=np.nan)

with torch.no_grad():
    for i in tqnb.tqdm(range(token_tensor.shape[0]-1)):
        edit_dist[flat_idxer[i]:flat_idxer[i+1]] = (token_tensor[i][None,:] != token_tensor[i+1:]).sum(dim=1)


  0%|          | 0/120056 [00:00<?, ?it/s]

In [8]:
torch.save({'condensed_distance': edit_dist}, 'hamming__condensed_distance_matrix.pt')

## redo shuffled

rerun this with a fresh kernel

In [1]:
import sys
import os
import shutil
import gzip
import csv
import multiprocessing

import tqdm.notebook as tqnb

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
import pandas as pd

import boda
from boda.common.utils import KmerFilter, dna2tensor

import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr

In [2]:
def pad_tensor_200(in_tensor):
    out_tensor = torch.cat([
        in_tensor,
        torch.zeros((4,200-in_tensor.shape[1]), device=in_tensor.device)
    ], dim=1)
    return out_tensor

In [3]:
seq_table = {'ID': [], 'sequence': []}

with open('boda_round_1_controls_20211215.shuffle_1.fa', 'r') as f:
    for i, line in enumerate(f):
        if i % 2 == 0:
            seq_table['ID'].append( line.rstrip().lstrip('>') )
        else:
            seq_table['sequence'].append( line.rstrip() )
            
seq_table = pd.DataFrame.from_dict(seq_table, orient='columns')

seq_tensor = torch.stack(
    [ pad_tensor_200(dna2tensor(line['sequence'])) for i, line in seq_table.iterrows() ], 
    dim=0
)

In [4]:
token_tensor = seq_tensor.argmax(dim=1)

In [5]:
section_size = torch.arange(token_tensor.shape[0]).flip(dims=[0])
flat_idxer   = torch.cat([torch.tensor([0],dtype=torch.long),torch.cumsum(section_size,dim=0,dtype=torch.long)])

edit_dist = torch.full((torch.arange(token_tensor.shape[0]).sum(),), fill_value=np.nan)

with torch.no_grad():
    for i in tqnb.tqdm(range(token_tensor.shape[0]-1)):
        edit_dist[flat_idxer[i]:flat_idxer[i+1]] = (token_tensor[i][None,:] != token_tensor[i+1:]).sum(dim=1)


  0%|          | 0/120056 [00:00<?, ?it/s]

In [6]:
torch.save({'condensed_distance': edit_dist}, 'hamming__condensed_distance_matrix.shuffle_1.pt')